In [ ]:
pip install pyarrow pandas dask

In [1]:
import os
import pyarrow.parquet as pq
import pandas as pd
from datetime import datetime

In [2]:
# Set your working directory to the folder containing your Parquet files
folder_path = "/content/drive/MyDrive/Applied Microeconometrics/data"
os.chdir(folder_path)

# Create a list of Parquet file paths in your folder
file_paths = [file for file in os.listdir() if file.endswith('.parquet')]

In [3]:
# Define the criteria for filtering
valid_PULocationID = [132, 138]
valid_DOLocationID = [229, 161, 162, 163, 164]

# Define the list of holiday dates (including 2014)
holiday_dates = [
    datetime(2011, 1, 1),  # New Year's Day
    datetime(2011, 4, 24),  # Easter Sunday
    datetime(2011, 5, 30),  # Memorial Day
    datetime(2011, 7, 4),   # Independence Day
    datetime(2011, 9, 5),   # Labor Day
    datetime(2011, 11, 24),  # Thanksgiving
    datetime(2011, 12, 25),  # Christmas Day
    datetime(2012, 1, 1),
    datetime(2012, 4, 8),
    datetime(2012, 5, 28),
    datetime(2012, 7, 4),
    datetime(2012, 9, 3),
    datetime(2012, 11, 22),
    datetime(2012, 12, 25),
    datetime(2013, 1, 1),
    datetime(2013, 3, 31),
    datetime(2013, 5, 27),
    datetime(2013, 7, 4),
    datetime(2013, 9, 2),
    datetime(2013, 11, 28),
    datetime(2013, 12, 25),
    datetime(2014, 1, 1),
    datetime(2014, 4, 20),
    datetime(2014, 5, 26),
    datetime(2014, 7, 4),
    datetime(2014, 9, 1),
    datetime(2014, 11, 27),
    datetime(2014, 12, 25)
]

In [ ]:
# # FULL DATASET
# full_merge = pd.DataFrame()

# # Loop through the Parquet files
# for file_path in file_paths:
#     # Read the Parquet file
#     parquet_data = pq.read_table(file_path).to_pandas()

#     # Combine the data with the existing data
#     full_merge = full_merge.append(parquet_data, ignore_index=True)


In [ ]:
# # Define the paths for the full panel Parquet and CSV files
# output_full_parquet_file = "/content/drive/MyDrive/Applied Microeconometrics/full_data.parquet"
# output_full_csv_file = "/content/drive/MyDrive/Applied Microeconometrics/full_data.csv"

In [ ]:
# # Export the full dataset to a Parquet file
# full_merge.to_parquet(output_full_parquet_file, engine="pyarrow")

In [ ]:
# # Export the full dataset to a CSV file
# full_merge.to_csv(output_full_csv_file, index=False)

In [4]:
merged_data = pd.DataFrame()

# Loop through the Parquet files
for file_path in file_paths:
    # Read the Parquet file
    parquet_data = pq.read_table(file_path).to_pandas()

    # Filter based on PULocationID and DOLocationID
    pulocation_filter = parquet_data['PULocationID'].isin(valid_PULocationID)
    dolocation_filter = parquet_data['DOLocationID'].isin(valid_DOLocationID)
    filtered_data_subset = parquet_data[pulocation_filter & dolocation_filter]

    # Filter based on dates to exclude holidays
    pickup_datetime = pd.to_datetime(filtered_data_subset['tpep_pickup_datetime'])
    dropoff_datetime = pd.to_datetime(filtered_data_subset['tpep_dropoff_datetime'])
    date_filter = ~pickup_datetime.dt.date.isin(holiday_dates) & ~dropoff_datetime.dt.date.isin(holiday_dates)
    filtered_data_subset = filtered_data_subset[date_filter]

    # Additional filtering criteria
    valid_entries_filter = (
        (filtered_data_subset['trip_distance'] > 0) & # Excludes zero distance anomalies
        (filtered_data_subset['passenger_count'] > 0) & # Excludes zero passenger anomalies
        (filtered_data_subset['fare_amount'] > 0) & # Excludes zero fare anomalies
        (filtered_data_subset['tolls_amount'] == 0) &  # Exclude trips with tolls
        # (filtered_data_subset['payment_type'] == 1) &  # Including only credit card payments
        # (filtered_data_subset['tip_amount'] == 0) & # Exclude trips with tips
        (filtered_data_subset['total_amount'] <= filtered_data_subset['total_amount'].mean() + 3 * filtered_data_subset['total_amount'].std()) &  # Exclude extreme values
        (filtered_data_subset['total_amount'] >= filtered_data_subset['total_amount'].mean() - 3 * filtered_data_subset['total_amount'].std())
    )
    filtered_data_subset = filtered_data_subset[valid_entries_filter]

    # Combine the filtered data with the existing data
    merged_data = merged_data.append(filtered_data_subset, ignore_index=True)


<ipython-input-4-ae78fe238f1a>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(filtered_data_subset, ignore_index=True)
<ipython-input-4-ae78fe238f1a>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(filtered_data_subset, ignore_index=True)
<ipython-input-4-ae78fe238f1a>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(filtered_data_subset, ignore_index=True)
<ipython-input-4-ae78fe238f1a>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(filtered_data_subset, ignore_index=True)
<ipython-input-4-ae78fe2

In [5]:
len(merged_data.index)

401454

In [ ]:
# Display the head() of the merged dataset
print(merged_data.head())

In [6]:
# Define the paths for the output Parquet and CSV files
output_parquet_file = "/content/drive/MyDrive/Applied Microeconometrics/sample_data.parquet"
output_csv_file = "/content/drive/MyDrive/Applied Microeconometrics/sample_data.csv"

In [7]:
# # Export the sample dataset to a Parquet file
merged_data.to_parquet(output_parquet_file, engine="pyarrow")

In [8]:
# # Export the sample dataset to a CSV file
merged_data.to_csv(output_csv_file, index=False)